In [1]:
import pandas as pd
from ftfy import fix_text
import re
import json
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
pd.set_option('display.max_colwidth', None)
df = pd.read_csv('./gtest_5.csv')

In [2]:
df1 = pd.read_csv('./gtest_4.csv')
df2 = pd.read_csv('./gtest_5.csv')
df3 = pd.read_csv('./gtest_6.csv')
df = pd.concat([df1, df2, df3], ignore_index=True)
df.to_csv('./gem25pro.csv', index=False)
len(df)

300

In [2]:
n = 2
df_splits = np.array_split(df, n)
for i, split in enumerate(df_splits):
    split.to_csv(f'o3_{i+1}.csv', index=False)

d:\Python\Lib\site-packages\numpy\_core\fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


In [3]:
def split_predict_column_method1(df):
    df_copy = df.copy()
    def extract_json(text):
        match = re.search(r'{[\s\S]*?}', text)
        if match:
            return json.loads(match.group(0))
        else:
            return None
    if df_copy['predict'].dtype == 'object':
        df_copy['predict'] = df_copy['predict'].apply(lambda x: extract_json(x) if isinstance(x, str) else x)
    json_df = pd.json_normalize(df_copy['predict'])
    result_df = pd.concat([df_copy.drop('predict', axis=1), json_df], axis=1)
    return result_df

df = pd.read_csv('./gem25proc_prompt.csv')
df = df.dropna()
df['predict'] = df['predict'].str.strip()
df = split_predict_column_method1(df)
df = df[df['labels'] == df['answer']]
df = df.drop(columns=['label', 'claim_clean', 'reference_clean'])

In [8]:
def split_predict_column_method1(df):
    df_copy = df.copy()
    def extract_json(text):
        match = re.search(r'{[\s\S]*?}', text)
        if match:
            return json.loads(match.group(0))
        else:
            return None
    if df_copy['predict'].dtype == 'object':
        df_copy['predict'] = df_copy['predict'].apply(lambda x: extract_json(x) if isinstance(x, str) else x)
    json_df = pd.json_normalize(df_copy['predict'])
    result_df = pd.concat([df_copy.drop('predict', axis=1), json_df], axis=1)
    return result_df

df = pd.read_csv('./o3-mhpc-prompt.csv')
df['predict'] = df['predict'].str.strip()
df = split_predict_column_method1(df)
label_mapping = {
    'Opposite_meaning_probability': 'Opposite meaning',
    'Misrepresentation_probability': 'Misrepresentation',
    'Related_but_unverifiable_probability': 'Related but unverifiable',
    'Entailment_probability': 'Entailment',
    'Entity_error_probability': 'Entity error',
    'Unrelated_and_unverifiable_probability': 'Unrelated and unverifiable',
    'Numeric_error_probability': 'Numeric error',
    'Missing_information_probability': 'Missing information'
}

# Tạo cột answer bằng cách lấy nhãn có xác suất cao nhất
df['answer'] = df[
    list(label_mapping.keys())
].idxmax(axis=1).map(label_mapping)
df = df[df['labels'] == df['answer']]
df = df.drop(columns=['label', 'claim_clean', 'reference_clean'])

In [9]:
from scipy.stats import entropy
def calculate_negative_entropy(row):
    """
    Tính negative entropy cho một hàng dữ liệu
    """
    # Lấy các cột xác suất
    prob_cols = [
        'Opposite_meaning_probability',
        'Misrepresentation_probability', 
        'Related_but_unverifiable_probability',
        'Entailment_probability',
        'Entity_error_probability',
        'Unrelated_and_unverifiable_probability',
        'Numeric_error_probability',
        'Missing_information_probability'
    ]
    
    # Lấy giá trị xác suất và chuyển đổi kiểu dữ liệu
    try:
        probabilities = []
        for col in prob_cols:
            if col in row.index:
                val = row[col]
                if pd.notna(val):  # Kiểm tra không phải NaN
                    val = float(val)  # Chuyển đổi sang float
                    if val > 0:  # Chỉ lấy giá trị > 0
                        probabilities.append(val)
        
        probabilities = np.array(probabilities, dtype=float)
        
        if len(probabilities) == 0 or np.sum(probabilities) == 0:
            return 0.0
        
        # Tính entropy thủ công để tránh lỗi
        ent = np.sum(probabilities * np.log2(probabilities))  # Thêm epsilon để tránh log(0)
        
        # Trả về negative entropy
        return ent
        
    except Exception as e:
        print(f"Lỗi tính entropy cho hàng: {e}")
        return 0.0
# Tính negative entropy cho tất cả các hàng
df['negative_entropy'] = df.apply(calculate_negative_entropy, axis=1)

df['negative_entropy'].describe()

count    161.000000
mean      -0.243367
std        0.399082
min       -1.685475
25%       -0.468996
50%        0.000000
75%        0.000000
max        0.000000
Name: negative_entropy, dtype: float64

In [5]:
df.loc[df['negative_entropy']>0]

,ID,labels,answer,Opposite_meaning_probability,Misrepresentation_probability,Related_but_unverifiable_probability,Entailment_probability,Entity_error_probability,Unrelated_and_unverifiable_probability,Numeric_error_probability,Missing_information_probability,negative_entropy
173,i_2271,Misrepresentation,Misrepresentation,1.23,1.23,1.23,1.23,1.23,1.23,1.23,1.23,2.938798


In [10]:
df2 = pd.read_csv('./subtask2_train_batch3.csv').drop(columns=['question', 'answer'])
df2['reference'] = df2['reference'].apply(fix_text)

In [11]:
df2['total_length'] = (
    df2['claim'].astype(str).str.len() +
    df2['reference'].astype(str).str.len() +
    df2['justification'].astype(str).str.len()
)

df_combined = df2.merge(df[['ID', 'labels', 'negative_entropy']], on='ID', how='left')
n = 1
df3 = (
    df_combined
    .sort_values(by=['labels', 'negative_entropy', 'total_length'], ascending=[True, False, True])
    .groupby('labels', group_keys=False)
    .head(n)
    .reset_index(drop=True)
)
df3.drop(columns=['ID', 'label', 'total_length', 'negative_entropy'], inplace=True)
#df3[df3['labels'] == 'Unrelated and unverifiable']
df3

,claim,reference,justification,labels
0,Measurement and Validation: 3D Laser Scanners: Employ advanced scanning techniques to locate and identify noise sources in complex acoustic environments [11].,"[11]: This paper describes the method for the acoustic analysis of noise sources in industry halls using new measurement technologies. Measurements were carried out by an acoustic camera (system using beamforming method) and 3D laser used for scanning closed-spaces geometry. The purpose of these measurements is to locate and identify noise sources on a production line in a complicated acoustic situation (reverberation conditions, many and various noise sources). The results of these measurements are used as an input in computer simulations with software for noise prediction which leads to creation of noise sources ranking and finding possibilities to reduce the noise level. The measurements show the potential of new measurement technologies of faster and more precise acoustic analysis of noise sources in industry halls. © European Acoustics Association.","Justification 1: The reference states that an acoustic camera and 3D laser are used to locate and identify noise sources in a complicated acoustic situation, which the claim indicates. > "" 3D laser used for scanning closed-spaces geometry. The purpose of these measurements is to locate and identify noise sources on a production line in a complicated acoustic situation""\n",Entailment
1,"Implementation Challenges: The United States faces challenges in coordinating the establishment of a CO2 transport network, which is essential for the wide implementation of CCS. This requires significant public planning and regulation [3].","[3]: Carbon dioxide capture and storage (CCS) has recently been receiving increasing recognition in policy debates. Various aspects of possible regulatory frameworks for its implementation are beginning to be discussed in Europe. One of the issues associated with the wide use of CCS is that it requires the establishment of a carbon dioxide (CO <inf>2</inf>) transport network, which could result in the spatial restructuring of power generation and transmission systems. This poses a significant coordination problem necessitating public planning and regulation. This paper provides a survey over multiple research strands on CCS, particularly energy system modeling and spatial optimization, pertaining to the efficient installment of CCS-related infrastructure throughout Europe. It integrates existing findings and highlights the factors that determine policy coordination needs for a potential wide implementation of CCS in the next decades. © 2012 Springer Science+Business Media B.V.",The EU faces challenges in coordinating establishment not the United States.,Entity error
2,"Key Challenges: Open Standards and Interoperability: The absence of open standards and interoperability between various smart city systems is likely to completely prevent any integration of services, thereby making them inaccessible to most citizens [1].","[1]: The holy grail of smart cities is an integrated, sustainable approach to improve the efficiency of the city's operations and the quality of life of citizens. At the heart of this vision is the citizen, who is the primary beneficiary of smart city initiatives, either directly or indirectly. Despite the recent surge of research and smart cities initiatives in practice, there are still a number of challenges to overcome in realizing this vision. This position paper points out six citizen-related challenges: the engagement of citizens, the improvement of citizens' data literacy, the pairing of quantitative and qualitative data, the need for open standards, the development of personal services, and the development of persuasive interfaces. The article furthermore advocates the use of methods and techniques from GIScience to tackle these challenges, and presents the concept of an Open City Toolkit as a way of transferring insights and solutio